In [1]:
import requests as r
from bs4 import BeautifulSoup
import geopandas as gpd
from shapely.geometry import Polygon,MultiPolygon
import re

In [2]:
def get_regs_names_hrefs(href):
    soup=BeautifulSoup(r.get(href).text,'lxml')
    distrs=[re.split(r'\s+ОКТМО',i.text)[0] for i in soup.find_all('li',attrs={'class':'col-md-6 col-12'})]
    refs=[i for i in re.findall(r'href="(.+?)"',str(soup.find_all('li',attrs={'class':'col-md-6 col-12'})))]
    return (distrs,refs)

In [3]:
def get_subregs(reg_tuple):
    total=[]
    distrs,refs=reg_tuple
    for index,distr in enumerate(distrs):
        soup=BeautifulSoup(r.get('https://oktmorus.ru'+refs[index]).text,'lxml')
        subdistrs=[ re.split(r'\s+ОКТМО',i.text)[0] for i in soup.find_all('ul')[1].find_all('li')]
        if subdistrs==[]:
            pass
        else:
            total.append((distr,subdistrs))
    return total

In [4]:
def geocode(tup):
    mun=r.get('http://nominatim.openstreetmap.org/search?',params={'state':tup[0],'county':tup[1],'format':'json','polygon_geojson':'1'})
    if mun.json()[0]['geojson']['type']=='MultiPolygon':
        pols=[]
        for obj in mun.json()[0]['geojson']['coordinates']:
            for po in obj:
                pols.append(Polygon(po))
        p=pols[0]
        pols1=pols[1:]
        new_p=[]
        for i in pols1:
            if p.intersects(i):
                p=p.symmetric_difference(i)
            else:
                k=0
                for ind,n in enumerate(new_p):
                    if n.intersects(i):
                        new_p[ind]=new_p[ind].symmetric_difference(i)
                        k=1
                        break
                        
                if k==0:
                    new_p.append(i)
        new_p.insert(0,p)
        try:
            return MultiPolygon(new_p)
        except:
            return new_p[0]
    else:
        geo_data=mun.json()[0]['geojson']['coordinates']
        mulp=[]
        if len(geo_data)==1:
            return Polygon(geo_data[0])
        else:
            for data in geo_data:
                mulp.append(Polygon(data))
            or_data=mulp[0]
            mulp=mulp[1:]
            for p in mulp:
                or_data=or_data.symmetric_difference(p)
            return or_data

In [5]:
def geodataframes(total):
    gdf_list=[]
    for tup in total:
        geom=[]
        distr,subd=tup
        print(f'Обрабатываю {distr}')
        for sd in subd:
            sd=sd.rstrip().lstrip().replace('город ',' ').replace('ЗАТО ',' ').replace('муниципальный ',' ')
            att=False
            while not att:
                try:
                    geom.append(geocode((distr,sd)))
                    att=True
                except:
                    print(f'Ошибка в {distr} и пункте {sd}.Перенаберите')
                    distr=input()
                    sd=input()
                    if '---' in distr:
                        pass
        gdf_list.append((distr,gpd.GeoDataFrame(subd,crs={'init': 'epsg:4326'},geometry=geom)))
    return gdf_list
                
    

In [6]:
rd=get_subregs(get_regs_names_hrefs('https://oktmorus.ru/privolzhskij-federalnyij-okrug/'))

In [ ]:
gdf_list=geodataframes(rd)

Обрабатываю Нижегородская область
Ошибка в Нижегородская область и пункте Кулебакский  район.Перенаберите
Нижегородская область
Кулебаки
Ошибка в Нижегородская область и пункте Семеновский.Перенаберите
Нижегородская область
Семёновский


C:\Users\timna\anaconda3\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Обрабатываю Кировская область
Ошибка в Кировская область и пункте Богородский  район.Перенаберите
Кировская область
Богородский
Ошибка в Кировская область и пункте Куменский  район.Перенаберите
Кировская область
Куменский
Ошибка в Кировская область и пункте Куменский.Перенаберите
Кировская область
Кумёнский район
Ошибка в Кировская область и пункте Санчурский  район.Перенаберите
Кировская область
Санчурский
Ошибка в Кировская область и пункте Фаленский  район.Перенаберите


In [ ]:
def save(gdf_list,folder):
    for gdf in gdf_list:
        gdf[1].columns=['район','geometry']
        gdf[1].to_file(folder+f'/{gdf[0]}.geojson',encoding='utf-8',driver='GeoJSON')